In [ ]:
`julia scripts/slurm/runners/runMrBayes.jl \
    -i data/sims/MSAs/gtdb-f__Burkholderiaceae-t128-l50000-b20.phy \
    -o _research/runMrBayes/gtdb-f__Burkholderiaceae-t128-l50000-b20 \
    -m WAG`

In [ ]:
`julia scripts/slurm/runners/runRAxML.jl \
    -i data/sims/MSAs/gtdb-f__Burkholderiaceae-t128-l50000-b20.phy \
    -o _research/runRAxML/gtdb-f__Burkholderiaceae-t128-l50000-b20 \
    -m WAG`

In [ ]:
`julia --threads=5 scripts/slurm/runners/runSPI.jl \
    -i data/sims/MSAs/gtdb-f__Burkholderiaceae-t128-l50000-b20.phy \
    -o _research/runSPI/gtdb-f__Burkholderiaceae-t128-l50000-b20`

In [ ]:
`julia scripts/slurm/runners/runFastTree.jl \
    -i data/sims/MSAs/gtdb-f__Burkholderiaceae-t128-l50000-b20.phy \
    -o _research/runFastTree/gtdb-f__Burkholderiaceae-t128-l50000-b20 \
    -m WAG`